# Validación del formulario del Método Primario en el EFAR

* Realizado por:
    * **José Abrahan Alfaro Alvarado**
* Instituto Tecnológico de Costa Rica
* Por motivo de: Herramienta VALIDACIÓN para Trabajo Final de Graduación
* Fecha: 8/11/2022

# Importación de las bibliotecas necesarias

In [1]:
# Se importan las librerias necesarias para el tratamiento de los datos
from IPython.display import Image
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
from numpy.random import normal
from numpy.random import uniform
from numpy.random import triangular


# Num de iteraciones
N = 10**7
n = 10 # Ciclos de cada corrida

La implementación del método primario de calibración del EFAR se compone de dos etapas:
* La primera: 
    * cuantificación de los errores de medida a partir de las lámparas primarias
    * cuantificación de incertidumbres
* La segunda:
    * cuantificación de las longitudes de onda promedio de transmitancia mínima medidas del SRM 2034
    * cuantificación de incertidumbres

# Algoritmos de funciones

## Funciones para la validación del primario

In [ ]:
def Selecciona_Parametros_Primario(ABE):
    '''
    Retorna los valores seleccionados para la validación en función del ABE
    
    Entrada:
    - ABE: valor del ABE que se quiere analizar.
    
    Salida: 
    - longitudes_Hg_NIST: array que contiene las longitudes de onda de referencia.
    - u_ABE: el valor de la incertidumbre asociada al ABE.
    - sr: array que contiene las repetibilidades en función del ABE.
    - sI: array que contiene las repetibilidades intermedias en función del ABE.
    - longitudes_promedio: array que contiene los valores de las longitudes de onda mancomunadas.
    '''
    # Se definen las longitudes de onda patrón de la lámpara de Hg según el NIST CODATA.
    longitudes_Hg_NIST = hoja_sr["Longitud de onda"].values
    longitudes_Hg_NIST = np.delete(longitudes_Hg_NIST, (1, 4, 5))
        
    # Se inician los condicionales.
    if ABE == 0.1:
        longitudes_Hg_NIST = hoja_sr["Longitud de onda"].values
        u_ABE = uniform(-s_max_ABE1,s_max_ABE1, size=N)
        sr = sr_ABE1
        sI = sI_ABE1
        longitudes_promedio = longitudes_promedio_ABE1
    elif ABE == 0.5:
        u_ABE = uniform(-s_max_ABE2,s_max_ABE2, size=N)
        sr = sr_ABE2
        sI = sI_ABE2
        longitudes_promedio = longitudes_promedio_ABE2
    elif ABE == 1:
        u_ABE = uniform(-s_max_ABE3,s_max_ABE3, size=N)
        sr = sr_ABE3
        sI = sI_ABE3
        longitudes_promedio = longitudes_promedio_ABE3
    elif ABE == 2:
        u_ABE = uniform(-s_max_ABE4,s_max_ABE4, size=N)
        sr = sr_ABE4
        sI = sI_ABE4
        longitudes_promedio = longitudes_promedio_ABE4
        
    return longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio

In [ ]:
def Valida_Formulario_Primario(longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio):
    
    # Ciclo for que calculará todos los valores a validadar
    u_sr = [] # Array que contendrá las distribuciones del aporte de sr para cada longitud de onda
    u_sI = [] # Array que contendrá las distribuciones del aporte de sI para cada longitud de onda
    u_reproducibilidad = [] # Array que contendrá las distribuciones del aporte de combinado de sr y sI para cada longitud de onda
    error_Enf1 = [] # Array que contendrá las distribuciones de los errores de medida finales para cada longitud de onda en el enfoque 1
    error_Enf2 = [] # Array que contendrá las distribuciones de los errores de medida finales para cada longitud de onda en el enfoque 2

    for ilongitud in range(len(longitudes_Hg_NIST)):
        #u_sr = normal(0,sr_ABE1[ilongitud]/np.sqrt(n), size=N)
        u_sr.append(normal(0,sr[ilongitud]/np.sqrt(n), size=N))

        #u_sI = normal(0,sI_ABE1[ilongitud], size=N)
        u_sI.append(normal(0,sI[ilongitud], size=N))

        u_reproducibilidad.append(u_sr[ilongitud]+u_sI[ilongitud])

        E = Error_de_medida(longitudes_promedio[ilongitud],longitudes_Hg_NIST[ilongitud])

        error_Enf1.append(E + u_res + u_ABE + u_sr[ilongitud])
        error_Enf2.append(E + u_res + u_ABE + u_reproducibilidad[ilongitud])

    error_promedio_Enf1 = []
    error_promedio_Enf2 = []
    incertidumbre_estandar_res = []
    incertidumbre_estandar_ABE = []
    incertidumbre_estandar_sr = []
    incertidumbre_estandar_reproducibilidad = []
    incertidumbre_estandar_comb_Enf1 =[]
    incertidumbre_estandar_comb_Enf2 = []

    for ilongitud in range(len(longitudes_Hg_NIST)):
        error_promedio_Enf1.append(error_Enf1[ilongitud].mean())
        error_promedio_Enf2.append(error_Enf2[ilongitud].mean())
        incertidumbre_estandar_res.append(u_res.std())
        incertidumbre_estandar_ABE.append(u_ABE.std())
        incertidumbre_estandar_sr.append(u_sr[ilongitud].std())
        incertidumbre_estandar_reproducibilidad.append(u_reproducibilidad[ilongitud].std())
        incertidumbre_estandar_comb_Enf1.append(error_Enf1[ilongitud].std())
        incertidumbre_estandar_comb_Enf2.append(error_Enf2[ilongitud].std())
    
    return error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2, u_sr, u_reproducibilidad 

In [ ]:
def Graficas_Primario(graficar, u_ABE, u_sr, u_reprod, error_Enf1, error_Enf2):
    '''
    Función que grafica los resultados de las simulaciones de Monte Carlo, y retorna el valor de la incertidumbre expandida
    de la distribución del error de medida incresado.
    
    Entradas: 
    - graficar: valor booleano que indica si se realizan las gráficas o no.
    - u_ABE: es la distribución que representa el aporte de la incertidumbre por ABE
    - u_sr: es la distribución que representa el aporte de la repetibilidad
    - u_reprod: es la distribución que representa el aporte de la repetibilidad y la repetibilidad intermedia = aporte de la reproducibilidad
    - error_Enf1: es la distribución final del error de medida considerando el enfoque 1
    - error_Enf2: es la distribución final del error de medida considerando el enfoque 2
    
    Salidas:
    - Gráficas
    '''
    if graficar:
        # Graficación
        plt.figure(figsize=(15,15))
        #----- FILA 1 -----#
        plt.subplot2grid((3,2), (0,0))
        plt.title(r"Resolución, ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(u_res, bins=200, density=True, color="maroon")

        plt.subplot2grid((3,2), (0,1))
        plt.title(r"ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(u_ABE, bins=200, density=True, color="indianred")
        #----- FILA 2 -----#
        plt.subplot2grid((3,2), (1,0))
        plt.title(r"Repetibilidad $s_{r}$, ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(u_sr, bins=200, density=True, color="darkblue")

        plt.subplot2grid((3,2), (1,1))
        plt.title(r"Reproducibilidad $s_{reproducibilidad}$, ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(u_reprod, bins=200, density=True, color="teal")
        #----- FILA 3 -----#
        plt.subplot2grid((3,2), (2,0))
        plt.title(r"Error de medida - Enfoque 1, ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(error_Enf1, bins=200, density=True, color="olivedrab")

        plt.subplot2grid((3,2), (2,1))
        plt.title(r"Error de medida - Enfoque 2, ABE = 0.1 nm, $\lambda$ = 253.66 nm")
        plt.ylabel("Frecuencia")
        plt.hist(error_Enf2, bins=200, density=True, color="yellowgreen")
        plt.show()
    return 

In [ ]:
def DataFrame_Final_Primario(k_Enf1, longitudes_Hg_NIST, error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2):
    # Generación del Data Frame Final
    longitudes_Hg_NIST = pd.DataFrame(longitudes_Hg_NIST, columns =["Longitud de onda"], dtype = float)
    error_promedio_Enf1 = pd.DataFrame(error_promedio_Enf1, columns = [r"$\bar{E}_{i}$ (nm), Enf 1"], dtype = float)
    incertidumbre_estandar_res = pd.DataFrame(incertidumbre_estandar_res, columns = [r"$u_{res}$ (nm)"], dtype = float)
    incertidumbre_estandar_ABE = pd.DataFrame(incertidumbre_estandar_ABE, columns = [r"$u_{ABE}$ (nm)"], dtype = float)
    incertidumbre_estandar_sr = pd.DataFrame(incertidumbre_estandar_sr, columns = [r"$u_{s_{r}}$ (nm)"], dtype = float)
    k_Enf1 = pd.DataFrame(k_Enf1, columns = [r"k"], dtype = float)
    incertidumbre_estandar_reproducibilidad = pd.DataFrame(incertidumbre_estandar_reproducibilidad, columns = [r"$u_{reprod}$ (nm)"], dtype = float)
    error_promedio_Enf2 = pd.DataFrame(error_promedio_Enf2, columns =[r"$\bar{E}_{i}$ (nm), Enf 2"], dtype = float)
    incertidumbre_estandar_comb_Enf1 = pd.DataFrame(incertidumbre_estandar_comb_Enf1, columns =[r"$u_{c}$ (nm), Enf 1"], dtype = float)
    incertidumbre_estandar_comb_Enf2 = pd.DataFrame(incertidumbre_estandar_comb_Enf2, columns =[r"$u_{c}$ (nm), Enf 2"], dtype = float)

    df_final = longitudes_Hg_NIST.copy()
    df_final[r"$\bar{E}_{i}$ (nm), Enf 1"] = error_promedio_Enf1.values.copy()
    df_final[r"$u_{res}$ (nm)"] = incertidumbre_estandar_res.values.copy()
    df_final[r"$u_{ABE}$ (nm)"] = incertidumbre_estandar_ABE.values.copy()
    df_final[r"$u_{s_{r}}$ (nm)"] = incertidumbre_estandar_sr.values.copy()
    df_final[r"$u_{c}$ (nm), Enf 1"] = incertidumbre_estandar_comb_Enf1.values.copy()
    df_final[r"k"] = k_Enf1.values.copy()
    df_final[r"$\bar{E}_{i}$ (nm), Enf 2"] = error_promedio_Enf2.values.copy()
    df_final["$u_{reprod}$ (nm)"] = incertidumbre_estandar_reproducibilidad.values.copy()
    df_final[r"$u_{c}$ (nm), Enf 2"] = incertidumbre_estandar_comb_Enf2.values.copy()
    
    return df_final

## Funciones para el SRM 2034

In [31]:
def Selecciona_Parametros_SRM_Vel_Baja(ABE):
    '''
    Retorna los valores seleccionados para la validación de las mediciones del SRM en función del ABE
    
    Entrada:
    - ABE: valor del ABE que se quiere analizar.
    
    Salida: 

    '''        
    # Se inician los condicionales.
    if ABE == 0.1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE1 # Selecciono las longitudes de onda patrón 
        # para un ABE = 0.1 nm
        longitudes_promedio_SRM_Vel_Baja = longitudes_promedio_SRM_Vel_Baja_ABE1 # Selecciono las mediciones
        # promedio del EFAR del SRM a un ABE = 0.1 nm
        u_ABE = uniform(-s_max_ABE1,s_max_ABE1, size=N) # Se define la incertidumbre asociada al ABE = 0.1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE1
        s_SRM_Vel_Baja = s_SRM_Vel_Baja_ABE1 # Se definen las desviaciones estándar de las mediciones
        # del SRM a un ABE = 0.1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE1 # Se define la incertidumbre combinada máxima a tomar 
        # para la incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE1
        Correccion = Correccion_ABE1 # Se define la corrección a aplicar a todas las mediciones 
        # del SRM 2034 para un ABE = 0.1 nm
    elif ABE == 0.5:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE2 # Selecciono las longitudes de onda patrón para 
        # un ABE = 0.5 nm
        longitudes_promedio_SRM_Vel_Baja = longitudes_promedio_SRM_Vel_Baja_ABE2 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 0.5 nm
        u_ABE = uniform(-s_max_ABE2,s_max_ABE2, size=N) # Se define la incertidumbre asociada al ABE = 0.5 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE2
        s_SRM_Vel_Baja = s_SRM_Vel_Baja_ABE2 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 0.5 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE2 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE2
        Correccion = Correccion_ABE2 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 0.5 nm
    elif ABE == 1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE3 # Selecciono las longitudes de onda patrón para 
        # un ABE = 1 nm
        longitudes_promedio_SRM_Vel_Baja = longitudes_promedio_SRM_Vel_Baja_ABE3 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 1 nm
        u_ABE = uniform(-s_max_ABE3,s_max_ABE3, size=N) # Se define la incertidumbre asociada al ABE = 1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE3
        s_SRM_Vel_Baja = s_SRM_Vel_Baja_ABE3 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE3 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE3
        Correccion = Correccion_ABE3 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 1 nm
    elif ABE == 2:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE4 # Selecciono las longitudes de onda patrón para 
        # un ABE = 2 nm
        longitudes_promedio_SRM_Vel_Baja = longitudes_promedio_SRM_Vel_Baja_ABE4 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 2 nm
        u_ABE = uniform(-s_max_ABE4,s_max_ABE4, size=N) # Se define la incertidumbre asociada al ABE = 2 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE4
        s_SRM_Vel_Baja = s_SRM_Vel_Baja_ABE4 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 2 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE4 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE4
        Correccion = Correccion_ABE4 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 2 nm
        
    return longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion

In [32]:
def Selecciona_Parametros_SRM_Vel_Med(ABE):
    '''
    Retorna los valores seleccionados para la validación de las mediciones del SRM en función del ABE
    
    Entrada:
    - ABE: valor del ABE que se quiere analizar.
    
    Salida: 

    '''        
    # Se inician los condicionales.
    if ABE == 0.1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE1 # Selecciono las longitudes de onda patrón 
        # para un ABE = 0.1 nm
        longitudes_promedio_SRM_Vel_Med = longitudes_promedio_SRM_Vel_Med_ABE1 # Selecciono las mediciones
        # promedio del EFAR del SRM a un ABE = 0.1 nm
        u_ABE = uniform(-s_max_ABE1,s_max_ABE1, size=N) # Se define la incertidumbre asociada al ABE = 0.1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE1
        s_SRM_Vel_Med = s_SRM_Vel_Med_ABE1 # Se definen las desviaciones estándar de las mediciones
        # del SRM a un ABE = 0.1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE1 # Se define la incertidumbre combinada máxima a tomar 
        # para la incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE1
        Correccion = Correccion_ABE1 # Se define la corrección a aplicar a todas las mediciones 
        # del SRM 2034 para un ABE = 0.1 nm
    elif ABE == 0.5:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE2 # Selecciono las longitudes de onda patrón para 
        # un ABE = 0.5 nm
        longitudes_promedio_SRM_Vel_Med = longitudes_promedio_SRM_Vel_Med_ABE2 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 0.5 nm
        u_ABE = uniform(-s_max_ABE2,s_max_ABE2, size=N) # Se define la incertidumbre asociada al ABE = 0.5 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE2
        s_SRM_Vel_Med = s_SRM_Vel_Med_ABE2 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 0.5 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE2 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE2
        Correccion = Correccion_ABE2 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 0.5 nm
    elif ABE == 1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE3 # Selecciono las longitudes de onda patrón para 
        # un ABE = 1 nm
        longitudes_promedio_SRM_Vel_Med = longitudes_promedio_SRM_Vel_Med_ABE3 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 1 nm
        u_ABE = uniform(-s_max_ABE3,s_max_ABE3, size=N) # Se define la incertidumbre asociada al ABE = 1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE3
        s_SRM_Vel_Med = s_SRM_Vel_Med_ABE3 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE3 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE3
        Correccion = Correccion_ABE3 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 1 nm
    elif ABE == 2:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE4 # Selecciono las longitudes de onda patrón para 
        # un ABE = 2 nm
        longitudes_promedio_SRM_Vel_Med = longitudes_promedio_SRM_Vel_Med_ABE4 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 2 nm
        u_ABE = uniform(-s_max_ABE4,s_max_ABE4, size=N) # Se define la incertidumbre asociada al ABE = 2 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE4
        s_SRM_Vel_Med = s_SRM_Vel_Med_ABE4 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 2 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE4 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE4
        Correccion = Correccion_ABE4 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 2 nm
        
    return longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion

In [34]:
def Selecciona_Parametros_SRM_Vel_Alt(ABE):
    '''
    Retorna los valores seleccionados para la validación de las mediciones del SRM en función del ABE
    
    Entrada:
    - ABE: valor del ABE que se quiere analizar.
    
    '''        
    # Se inician los condicionales.
    if ABE == 0.1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE1 # Selecciono las longitudes de onda patrón 
        # para un ABE = 0.1 nm
        longitudes_promedio_SRM_Vel_Alt = longitudes_promedio_SRM_Vel_Alt_ABE1 # Selecciono las mediciones
        # promedio del EFAR del SRM a un ABE = 0.1 nm
        u_ABE = uniform(-s_max_ABE1,s_max_ABE1, size=N) # Se define la incertidumbre asociada al ABE = 0.1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE1
        s_SRM_Vel_Alt = s_SRM_Vel_Alt_ABE1 # Se definen las desviaciones estándar de las mediciones
        # del SRM a un ABE = 0.1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE1 # Se define la incertidumbre combinada máxima a tomar 
        # para la incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE1
        Correccion = Correccion_ABE1 # Se define la corrección a aplicar a todas las mediciones 
        # del SRM 2034 para un ABE = 0.1 nm
    elif ABE == 0.5:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE2 # Selecciono las longitudes de onda patrón para 
        # un ABE = 0.5 nm
        longitudes_promedio_SRM_Vel_Alt = longitudes_promedio_SRM_Vel_Alt_ABE2 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 0.5 nm
        u_ABE = uniform(-s_max_ABE2,s_max_ABE2, size=N) # Se define la incertidumbre asociada al ABE = 0.5 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE2
        s_SRM_Vel_Alt = s_SRM_Vel_Alt_ABE2 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 0.5 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE2 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE2
        Correccion = Correccion_ABE2 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 0.5 nm
    elif ABE == 1:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE3 # Selecciono las longitudes de onda patrón para 
        # un ABE = 1 nm
        longitudes_promedio_SRM_Vel_Alt = longitudes_promedio_SRM_Vel_Alt_ABE3 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 1 nm
        u_ABE = uniform(-s_max_ABE3,s_max_ABE3, size=N) # Se define la incertidumbre asociada al ABE = 1 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE3
        s_SRM_Vel_Alt = s_SRM_Vel_Alt_ABE3 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 1 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE3 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE3
        Correccion = Correccion_ABE3 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 1 nm
    elif ABE == 2:
        longitudes_patron_SRM = longitudes_patron_SRM_ABE4 # Selecciono las longitudes de onda patrón para 
        # un ABE = 2 nm
        longitudes_promedio_SRM_Vel_Alt = longitudes_promedio_SRM_Vel_Alt_ABE4 # Selecciono las mediciones 
        # promedio del EFAR del SRM a un ABE = 2 nm
        u_ABE = uniform(-s_max_ABE4,s_max_ABE4, size=N) # Se define la incertidumbre asociada al ABE = 2 nm
        U_longitudes_patron_SRM = U_longitudes_patron_SRM_ABE4
        s_SRM_Vel_Alt = s_SRM_Vel_Alt_ABE4 # Se definen las desviaciones estándar de las mediciones del 
        # SRM a un ABE = 2 nm
        uComb_Max_Enf1 = uComb_Max_Enf1_ABE4 # Se define la incertidumbre combinada máxima a tomar para la 
        # incertidumbre del error primario
        uComb_Max_Enf2 = uComb_Max_Enf2_ABE4
        Correccion = Correccion_ABE4 # Se define la corrección a aplicar a todas las mediciones del SRM 2034 
        # para un ABE = 2 nm
        
    return longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion

In [22]:
def Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM, u_ABE, U_longitudes_patron_SRM, s_SRM, uComb_Max, Correccion):

    # Ciclo for que calculará todos los valores a validadar
    u_rep_SRM = [] # Array que contendrá las distribuciones del aporte de la repetibilidad

    longitud_SRM = [] # Array que contendrá las distribuciones de las bandas de %T_min de medida 
    # finales en el enfoque 1.1

    u_Error_Max = [] # Array que contendrá los aportes del error primario de las bandas de %T_min 
    # de medida finales en el enfoque 1.1
    
    for ilongitud in range(len(longitudes_patron_SRM)):

        u_rep_SRM.append(normal(0,s_SRM[ilongitud]/np.sqrt(n), size=N))

        u_Error_Max.append(uniform(-2*uComb_Max,2*uComb_Max, size=N))

        long = longitudes_promedio_SRM[ilongitud] + Correccion

        longitud_SRM.append(long + u_res + u_ABE + u_rep_SRM[ilongitud] +  u_Error_Max[ilongitud])
        
    longitud_promedio_SRM_Corregida = []
    C_ABE = []
    incertidumbre_estandar_res = []
    incertidumbre_estandar_ABE = []
    incertidumbre_estandar_rep = []
    incertidumbre_estandar_error = []
    incertidumbre_estandar_comb_Enf11 =[]

    for ilongitud in range(len(longitudes_patron_SRM)):
        
        longitud_promedio_SRM_Corregida.append(longitud_SRM[ilongitud].mean())
        C_ABE.append(Correccion)
        incertidumbre_estandar_res.append(u_res.std())
        incertidumbre_estandar_ABE.append(u_ABE.std())
        incertidumbre_estandar_rep.append(u_rep_SRM[ilongitud].std())
        incertidumbre_estandar_error.append(u_Error_Max[ilongitud].std())
        incertidumbre_estandar_comb_Enf11.append(longitud_SRM[ilongitud].std())

    return longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11

In [23]:
def Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM, u_ABE, U_longitudes_patron_SRM, s_SRM, s_ajuste, m, b):

    # Ciclo for que calculará todos los valores a validadar
    u_rep_SRM = [] # Array que contendrá las distribuciones del aporte de la repetibilidad

    longitud_SRM = [] # Array que contendrá las distribuciones de las bandas de %T_min de medida finales en el enfoque 1.1

    u_ajuste = [] # Array que contendrá los aportes del error primario de las bandas de %T_min de medida finales en el enfoque 1.1
    
    CorreccionAjuste = []
    
    for ilongitud in range(len(longitudes_patron_SRM)):

        u_rep_SRM.append(normal(0,s_SRM[ilongitud]/np.sqrt(n), size=N))

        u_ajuste.append(normal(0, s_ajuste, size=N))
        
        CorreccionAjuste.append(-1*Ajuste_Error(m,b,longitudes_promedio_SRM[ilongitud]))

        long = longitudes_promedio_SRM[ilongitud] + CorreccionAjuste[ilongitud]
        longitud_SRM.append(long + u_res + u_ABE + u_rep_SRM[ilongitud] +  u_ajuste[ilongitud])
        
    longitud_promedio_SRM_Corregida = []
    incertidumbre_estandar_res = []
    incertidumbre_estandar_ABE = []
    incertidumbre_estandar_rep = []
    incertidumbre_estandar_ajuste = []
    incertidumbre_estandar_comb_Enf12 =[]

    for ilongitud in range(len(longitudes_patron_SRM)):
        
        longitud_promedio_SRM_Corregida.append(longitud_SRM[ilongitud].mean())
        incertidumbre_estandar_res.append(u_res.std())
        incertidumbre_estandar_ABE.append(u_ABE.std())
        incertidumbre_estandar_rep.append(u_rep_SRM[ilongitud].std())
        incertidumbre_estandar_ajuste.append(u_ajuste[ilongitud].std())
        incertidumbre_estandar_comb_Enf12.append(longitud_SRM[ilongitud].std())

    return longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12

In [24]:
def Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM, u_ABE, U_longitudes_patron_SRM, s_SRM, ucomb_max_enf2):

    # Ciclo for que calculará todos los valores a validadar
    u_rep_SRM = [] # Array que contendrá las distribuciones del aporte de la repetibilidad

    longitud_SRM = [] # Array que contendrá las distribuciones de las bandas de %T_min de medida finales en el enfoque 1.1
    
    u_comb_max_enf2 = [] 
    
    for ilongitud in range(len(longitudes_patron_SRM)):

        u_rep_SRM.append(normal(0,s_SRM[ilongitud]/np.sqrt(n), size=N))

        u_comb_max_enf2.append(normal(0, ucomb_max_enf2, size=N))
        
        long = longitudes_promedio_SRM[ilongitud]
        longitud_SRM.append(long + u_res + u_ABE + u_rep_SRM[ilongitud] +  u_comb_max_enf2[ilongitud])
        
    longitud_promedio_SRM = []
    CorreccionAjuste = []
    incertidumbre_estandar_res = []
    incertidumbre_estandar_ABE = []
    incertidumbre_estandar_rep = []
    incertidumbre_estandar_u_comb_max_enf2 = []
    incertidumbre_estandar_comb_Enf2 =[]
    
    for ilongitud in range(len(longitudes_patron_SRM)):
        
        longitud_promedio_SRM.append(longitud_SRM[ilongitud].mean())
        CorreccionAjuste.append(0)
        incertidumbre_estandar_res.append(u_res.std())
        incertidumbre_estandar_ABE.append(u_ABE.std())
        incertidumbre_estandar_rep.append(u_rep_SRM[ilongitud].std())
        incertidumbre_estandar_u_comb_max_enf2.append(ucomb_max_enf2)
        incertidumbre_estandar_comb_Enf2.append(longitud_SRM[ilongitud].std())
        
    return longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2

In [25]:
def DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitudes_promedio_SRM_Corregida, Correccion, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb):
    # Generación del Data Frame Final
    longitudes_patron_SRM = pd.DataFrame(longitudes_patron_SRM, columns =[r"Longitud de onda de ref (nm)"], dtype = float)
    longitudes_promedio_SRM_Corregida = pd.DataFrame(longitudes_promedio_SRM_Corregida, columns =[r"Longitud de onda EFAR (nm)"], dtype = float)
    C = pd.DataFrame(Correccion, columns =[r"Corrección (nm)"], dtype = float)
    incertidumbre_estandar_res = pd.DataFrame(incertidumbre_estandar_res, columns = [r"$u_{res}$ (nm)"], dtype = float)
    incertidumbre_estandar_ABE = pd.DataFrame(incertidumbre_estandar_ABE, columns = [r"$u_{ABE1}$ (nm)"], dtype = float)
    incertidumbre_estandar_rep = pd.DataFrame(incertidumbre_estandar_rep, columns = [r"$u_{rep}$ (nm)"], dtype = float)
    incertidumbre_estandar_error = pd.DataFrame(incertidumbre_estandar_error, columns = [r"$u_{error}$ (nm)"], dtype = float)
    incertidumbre_estandar_comb = pd.DataFrame(incertidumbre_estandar_comb, columns =[r"$u_{c}$ (nm)"], dtype = float)
    k_frame =  pd.DataFrame(k, columns =[r"Factor de cobertura, k"], dtype = float)
    E_N = pd.DataFrame(E_N, columns=[r"$E_{N}$"])

    df_final = longitudes_patron_SRM.copy()
    df_final[r"Longitud de onda EFAR (nm)"] = longitudes_promedio_SRM_Corregida.values.copy()
    df_final[r"Corrección"] = C.values.copy()
    df_final[r"$u_{res}$ (nm)"] = incertidumbre_estandar_res.values.copy()
    df_final[r"$u_{ABE}$ (nm)"] = incertidumbre_estandar_ABE.values.copy()
    df_final[r"$u_{rep}$ (nm)"] = incertidumbre_estandar_rep.values.copy()
    df_final[r"$u_{error}$ (nm)"] = incertidumbre_estandar_error.values.copy()
    df_final[r"$u_{c}$ (nm)"] = incertidumbre_estandar_comb.values.copy()
    df_final[r"Factor de cobertura, k"] = k_frame.values.copy()
    df_final[r"$E_{N}$"] = E_N.values.copy()
    
    return df_final

## Funciones globales

In [26]:
def Error_de_medida(longitud_onda_EFAR,longitud_onda_ref):
    '''
    Función que calcula el error de medida del EFAR
    
    Entradas:
    - longitud_onda_EFAR: el promedio mancomunado de las longitudes de onda medidas.
    - longitud_onda_Hg: valor de referencia patrón para la línea de emisión. Coincide con los valores reportados en CODATA
    
    Salida:
    - error_de_medida
    '''
    
    error_de_medida = longitud_onda_EFAR - longitud_onda_ref
    
    return error_de_medida

In [27]:
def Calculo_k(mensurando, u_comb, confianza):
    '''
    Función que a apartir de las distribuciones trapezoidales de los errores de medida del EFAR en función del ABE
    calcula la incertidumbre expandida y el factor de cobertura asociado. Esta función está diseñada para calcular 
    los factores de cobertura de los enfoques 1, los cuales presentan comportamientos trapezoidales.
    
    Entradas:
    - arreglo: arreglo con las distribuciones asociadas al mensurando bajo estudio.
    - u_comb: es el arreglo que contiene las incertidumbres estándar combinadas de dichas distribuciones de error.
    - confianza: es el nivel de confianza al que se cuantificará el intervalo de cobertura (varía de 0 a 100).
    
    Salidas:
    - U: arreglos con las incertidumbres expandidas de las distribuciones
    - factor_cobertura: es el factor de cobertura con el que se expanden las incertidumbres estándar combinadas.
    '''
    
    # Se define el nivel de significancia
    alpha = 1-confianza
    
    # Se crea un array vacío que contendrá los valores de las incertidumbres expandidas
    U = []
    
    # Se crea un array vacío que contendrá los valores de los factores de cobertura calculados
    factor_cobertura = []
    
    # Se inicia el ciclo for que iterará para cada una de las distribuciones del error 
    # de medida en función de la longitud de onda
    for ilongitud in range(len(mensurando)):
        # Se crea la variable que cuantifica el límite superior del intervalo de confianza 
        # del percentil 100-(alpha*100)/2
        lim_sup_interval_conf = np.percentile(mensurando[ilongitud], 100-(alpha*100)/2)
        # Adjuntamos la resta del valor de dicho límite superior y el valor promedio del 
        # estimado de error de medida para obtener la U
        U.append(lim_sup_interval_conf-mensurando[ilongitud].mean())
        # Adjuntamos el factor de cobertura como k = U/u_comb
        factor_cobertura.append(U[ilongitud]/u_comb[ilongitud])
        
    return np.round(factor_cobertura, 2)

In [28]:
def E_Normalizado(long_LCM, long_NIST, u_comb_LCM, U_NIST):
    '''
    Calcula los valores de error normalizado de las mediciones del EFAR
    
    Entrada:
    - long_LCM: es un arreglo que contiene las longitudes de onda promedio resultantes de las mediciones del EFAR.
    - long_NIST: es un arreglo que contiene las longitudes de onda certificadas por el NIST.
    - u_comb_LCM: es un arreglo que contiene las incertidumbres estándar combinadas de las long_LCM.
    - U_NIST: es un arreglo que contiene las incertidumbres expandidas certificadas por el NIST.
    
    Salida:
    - E_N: es un arreglo que contiene los errores normalizados de las mediciones del EFAR.
    '''
    
    E_N = [] # Se define el arreglo inicial.
    # Inicia el ciclo for que recorrerá cada longitud de onda
    for ilongitud in range(len(long_NIST)):
        # Se calcula el E_N tomando la suposición de que los k = 2
        E = (long_LCM[ilongitud] - long_NIST[ilongitud])/(2*np.sqrt(u_comb_LCM[ilongitud]**2+(U_NIST/2)**2))
        E_N.append(E[ilongitud])
        
    
    return np.round(E_N, 3)

In [29]:
def Ajuste_Error(m,b,ilongitud):
    '''
    Función que devuelve el valor del error de medida del EFAR en función de los parámetros
    de ajuste OLS del primario según el Enfoque 1.2.
    
    Entradas:
    - m: valor de la pendiente de la recta según los valores del ajuste.
    - b: valor de la intersección de la recta con el eje 'y' según los valores del ajuste.
    - ilongitud: valor de la longitud de onda donde se evaluará el ajuste.
    
    Salidas:
    - E: valor del error de medida a partir de la curva de ajuste OLS.
    '''
    E = m * ilongitud + b
    
    return E

In [11]:
# Defino el aporte de la resolución
r = 0.01 # nm resolución del equipo
u_res = uniform(-r/2,r/2, size=N) # Se divide entre dos debido a que r es el intervalo completo

# Primera etapa: cuantificación de los errores de medida que surgen del método primario

La determinación del error de medida del EFAR se obtiene mediante el siguiente modelo:

$$
E_{i} = \left(\bar{\lambda_{i}}-\lambda_{Hg}\right)+\epsilon_{rep}
$$
donde:
* $E_{i}$ es el error de medida del EFAR en la i-ésima longitud de onda de prueba.
* $\bar{\lambda_{i}}$ es el promedio de los promedios diarios de las mediciones del EFAR en la i-ésima longitud de onda de prueba.
* $\lambda_{Hg}$ es el valor de la longitud de onda de emisión de la lámpara tipo lápiz de mercurio.
* $\epsilon_{rep}$ es el aporte del error por repetibilidad que se asocia al modelo del error.

El diagrama de Ishikawa muestra las siguientes fuentes de incertidumbre para cada magnitud de entrada del modelo:

In [ ]:
Ishikawa_1 = r"C:\Users\Abraham Alfaro\Documents/TEC/2022/II SEMESTRE/TFG/Códigos en Jupyter para procesamiento de datos/Img/Ishikawa Primario.PNG"
Image(filename = Ishikawa_1, width=500, height=500)

Para $\bar{\lambda_{i}}$ se tiene que:
$$
\boxed{u_{res} = \frac{d}{\sqrt{12}}}
$$

$$
d = 0,01\ \text{nm}
$$

$$
\boxed{u_{ABE} = \frac{s_{ponderada_{máxima}}}{\sqrt{3}}}
$$
Para $\epsilon_{rep}$ se tienen dos enfoques:
* Enfoque 1:
$$
\boxed{u_{s_{r}}= \frac{s_{r}}{\sqrt{n}}}
$$
* Enfoque 2:
$$
\boxed{u_{s_{I}}= s_I}
$$

Para $\lambda_{Hg}$ se tiene que:
$$
\boxed{u_{\lambda_{Hg}}=0}
$$

## Importación de datos

In [12]:
# Se importan los datos en crudo necesarios para la validación de la hoja de cálculo
path = r"C:\Users\Abraham Alfaro\Documents\TEC\2022\II SEMESTRE\TFG\Entregables\Documentación - Protocolos - Hojas de Cálculo\Hojas de cálculo\Base_de_datos_validación_GS_SMI_FM_01.xlsx"
hoja_longitudes = pd.read_excel(path,sheet_name="Longitudes Onda Promedio",header=0)
hoja_DesABE = pd.read_excel(path,sheet_name="Desviaciones ABE Max Promedio",header=0)
hoja_sr = pd.read_excel(path,sheet_name="sr",header=0)
hoja_sI = pd.read_excel(path,sheet_name="sI",header=0)

In [13]:
# Se definen los arrays que contendrán los promedios mancomunados de las longitudes de onda en función del ABE
# Estos valores se utilizan en la validación del primario
longitudes_promedio_ABE1 = hoja_longitudes["ABE = 0.1 nm"].values
longitudes_promedio_ABE2 = hoja_longitudes["ABE = 0.5 nm"].dropna().values
longitudes_promedio_ABE3 = hoja_longitudes["ABE = 1 nm"].dropna().values
longitudes_promedio_ABE4 = hoja_longitudes["ABE = 2 nm"].dropna().values

In [14]:
# Se definen los arrays que contendrán las desviaciones ponderadas máximas del ABE
# Estos valores se utilizan a lo largo de toda la validación
s_max_ABE1 = hoja_DesABE["ABE = 0.1 nm"][0]
s_max_ABE2 = hoja_DesABE["ABE = 0.5 nm"][0]
s_max_ABE3 = hoja_DesABE["ABE = 1 nm"][0]
s_max_ABE4 = hoja_DesABE["ABE = 2 nm"][0]

In [15]:
# Se definen los arrays que contendrán las desviaciones por repetibilidad dentro de los días en función del ABE
# Estos valores se utilizan en la validación del primario
sr_ABE1 = hoja_sr["ABE = 0.1 nm"].values
sr_ABE2 = hoja_sr["ABE = 0.5 nm"].dropna().values
sr_ABE3 = hoja_sr["ABE = 1 nm"].dropna().values
sr_ABE4 = hoja_sr["ABE = 2 nm"].dropna().values

In [16]:
# Se definen los arrays que contendrán las desviaciones por repetibilidad intermedia en función del ABE
# Estos valores se utilizan en la validación del primario
sI_ABE1 = hoja_sI["ABE = 0.1 nm"].values
sI_ABE2 = hoja_sI["ABE = 0.5 nm"].dropna().values
sI_ABE3 = hoja_sI["ABE = 1 nm"].dropna().values
sI_ABE4 = hoja_sI["ABE = 2 nm"].dropna().values

## Validación del los resultados del Método Primario

### ABE = 0,1 nm

In [ ]:
# Defino el ABE a validar
ABE  = 0.1

# Selecciono los parámetros correspondientes
longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio = Selecciona_Parametros_Primario(ABE)

# Cálculo de todos los valores a validar
error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2, u_sr, u_reproducibilidad = Valida_Formulario_Primario(longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio) 

# Cálculo del factor de cobertura para las distribuciones trapezoidales del Enfoque 1
k_Enf1 = Calculo_k(error_Enf1,incertidumbre_estandar_comb_Enf1, 0.9545)

# Cálculo del Data Frame Final de los primarios
df_final_ABE1 = DataFrame_Final_Primario(k_Enf1, longitudes_Hg_NIST, error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2)

In [ ]:
# Graficación
ilongitud = 0
Graficas_Primario(False, u_ABE, u_sr[ilongitud], u_reproducibilidad[ilongitud], error_Enf1[ilongitud], error_Enf2[ilongitud])

### ABE = 0,5 nm

In [ ]:
# Defino el ABE a validar
ABE  = 0.5

# Selecciono los parámetros correspondientes
longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio = Selecciona_Parametros_Primario(ABE)

# Cálculo de todos los valores a validar
error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2, u_sr, u_reproducibilidad = Valida_Formulario_Primario(longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio) 

# Cálculo del factor de cobertura para las distribuciones trapezoidales del Enfoque 1
k_Enf1 = Calculo_k(error_Enf1,incertidumbre_estandar_comb_Enf1, 0.9545)

df_final_ABE2 = DataFrame_Final_Primario(k_Enf1, longitudes_Hg_NIST, error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2)

In [ ]:
# Graficación
ilongitud = 0
Graficas_Primario(False, u_ABE, u_sr[ilongitud], u_reproducibilidad[ilongitud], error_Enf1[ilongitud], error_Enf2[ilongitud])

### ABE = 1 nm

In [ ]:
# Defino el ABE a validar
ABE  = 1

# Selecciono los parámetros correspondientes
longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio = Selecciona_Parametros_Primario(ABE)

# Cálculo de todos los valores a validar
error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2, u_sr, u_reproducibilidad = Valida_Formulario_Primario(longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio) 

# Cálculo del factor de cobertura para las distribuciones trapezoidales del Enfoque 1
k_Enf1 = Calculo_k(error_Enf1,incertidumbre_estandar_comb_Enf1, 0.9545)

df_final_ABE3 = DataFrame_Final_Primario(k_Enf1, longitudes_Hg_NIST, error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2)

In [ ]:
# Graficación
ilongitud = 0
Graficas_Primario(False, u_ABE, u_sr[ilongitud], u_reproducibilidad[ilongitud], error_Enf1[ilongitud], error_Enf2[ilongitud])

### ABE = 2 nm

In [ ]:
# Defino el ABE a validar
ABE  = 2

# Selecciono los parámetros correspondientes
longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio = Selecciona_Parametros_Primario(ABE)

# Cálculo de todos los valores a validar
error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2, u_sr, u_reproducibilidad = Valida_Formulario_Primario(longitudes_Hg_NIST, u_ABE, sr, sI, longitudes_promedio) 

# Cálculo del factor de cobertura para las distribuciones trapezoidales del Enfoque 1
k_Enf1 = Calculo_k(error_Enf1,incertidumbre_estandar_comb_Enf1, 0.9545)

df_final_ABE4 = DataFrame_Final_Primario(k_Enf1, longitudes_Hg_NIST, error_Enf1, error_Enf2, error_promedio_Enf1, error_promedio_Enf2, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_sr, incertidumbre_estandar_reproducibilidad, incertidumbre_estandar_comb_Enf1, incertidumbre_estandar_comb_Enf2)

In [ ]:
# Graficación
ilongitud = 0
Graficas_Primario(False, u_ABE, u_sr[ilongitud], u_reproducibilidad[ilongitud], error_Enf1[ilongitud], error_Enf2[ilongitud])

### Resultados de validación

In [ ]:
df_final_ABE1

In [ ]:
df_final_ABE2

In [ ]:
df_final_ABE3

In [ ]:
df_final_ABE4

# Segunda etapa: cuantificación de las longitudes de onda promedio de transmitancia mínima medidas del SRM 2034

Esta segunda etapá se compone de tres sub etapas distintas para cada rapidez de barrido y para cada ABE. Cada una determina por el cómo se toman en consideración los resultados del procedimiento primario de la primera etapa.
* La sub-etapa 1 cuantifica las longitudes de transmitancia mínima tomando en consideración una corrección resultante de los promedios de los errores de medida primarios a partir del enfoque 1 así como la incertidumbre estándar combinada máxima dependiendo del ABE.
* La sub-etapa 2 cuantifica las longitudes de transmitancia mínima tomando en consideración un ajuste OLS a partir de los resultados del primario y la incertidumbre global asociada a dicho ajuste.
* La sub-etapa 3 cuantifica las longitudes de transmitancia mínima tomando en consideración el NO corregir, y el tomar la incertidumbre estádar combinada máxima (del enfoque 2) de los errores de medida del ABE respectivo como la fuente de incertidumbre por el error primario.

## Rapidez de barrido baja: 14.91 nm/min

### Sub-Etapa 1: Análisis del enfoque 1, corrrigiendo y tomando la incertidumbre estándar combinada máxima del error de medida en función del ABE.

La determinación de las longitudes de onda de transmitancia mínima del filtro de óxido de holmio SRM 2034, se obtiene mediante el siguiente 
modelo:

$$
\lambda_{EFAR_{i}} = \bar{\lambda_{i}}+C_{error}
$$
donde:
* $\lambda_{EFAR_{i}}$ es la medición del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $\bar{\lambda_{i}}$ es el promedio de las mediciones del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $C_{error}$ es la corrección originda por los datos del método primario de la primera etapa.

El diagrama de Ishikawa muestra las siguientes fuentes de incertidumbre para cada magnitud de entrada del modelo:

In [ ]:
Ishikawa_2 = r"C:\Users\Abraham Alfaro\Documents/TEC/2022/II SEMESTRE/TFG/Códigos en Jupyter para procesamiento de datos/Img/Ishikawa_VelBaja_1.1.PNG"
Image(filename = Ishikawa_2, width=500, height=500)

Para $\bar{\lambda_{i}}$ se tiene que:
$$
\boxed{u_{res} = \frac{d}{\sqrt{12}}}
$$

$$
d = 0,01\ \text{nm}
$$

$$
\boxed{u_{ABE} = \frac{s_{ponderada_{máxima}}}{\sqrt{3}}}
$$

$$
\boxed{u_{rep} = \frac{s_{\lambda_{i}}}{\sqrt{n}}}
$$
Para $C_{error}$ se tiene que:
$$
\boxed{u_{Error_{máx}}=\frac{U_{Error_{máx}}}{\sqrt{3}}}
$$

#### Importación de datos

In [36]:
# Se importan los datos en crudo necesarios para la validación de la hoja de cálculo en la segunda etapa

# Se importan los valores de longitudes promedio medidos del SRM a una velocidad baja
hoja_longitudes_SRM_Vel_Baja = pd.read_excel(path,sheet_name="Longitudes Promedio SRM VelBaja",header=0)

# Se importan los valores de las correcciones para el enfoque 1.1 (Las mismas para todas las RB)
hoja_correcciones_SRM_Enf11 = pd.read_excel(path,sheet_name="Corrección SRM Enf1.1",header=0)

# Se importan los valores de las desviaciones estándar obtenidas con la RB baja
hoja_Des_SRM_Vel_Baja = pd.read_excel(path,sheet_name="s SRM 2034 Vel Baj",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 1 (Primario) para Enf 1.1
hoja_uComb_Primario_Max_Enf1 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 1",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 2 (Primario) para Enf 2
hoja_uComb_Primario_Max_Enf2 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 2",header=0)

# Se importan los valores del certificado del SRM 2034
hoja_Certificado_SRM2034 = pd.read_excel(path,sheet_name="Certificado SRM 2034",header=0)

In [37]:
# Se definen los arrays que contendrán los promedios mancomunados de las longitudes de onda de 
# transmitancia mínima en función del ABE

# Se definen las longitudes de onda promedio para: ABE = 0.1 nm y RB = 14.91 nm/min
longitudes_promedio_SRM_Vel_Baja_ABE1 = hoja_longitudes_SRM_Vel_Baja["ABE = 0.1 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 0.5 nm y RB = 14.91 nm/min
longitudes_promedio_SRM_Vel_Baja_ABE2 = hoja_longitudes_SRM_Vel_Baja["ABE = 0.5 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 1 nm y RB = 14.91 nm/min
longitudes_promedio_SRM_Vel_Baja_ABE3 = hoja_longitudes_SRM_Vel_Baja["ABE = 1 nm"].values

# Se definen las longitudes de onda promedio para: ABE = 2 nm y RB = 14.91 nm/min
longitudes_promedio_SRM_Vel_Baja_ABE4 = hoja_longitudes_SRM_Vel_Baja["ABE = 2 nm"].values

In [38]:
# Se definen los arrays que contendrán los valores certificados del NIST de las bandas de transmitancia 
# mínima del SRM 2034

# Se definen los valores de las bandas de %T mínima para un ABE = 0.1 nm desde el Certificado del NIST
longitudes_patron_SRM_ABE1 = hoja_Certificado_SRM2034["ABE = 0.1 nm"].dropna().values

# Se definen los valores de las bandas de %T mínima para un ABE = 0.5 nm desde el Certificado del NIST
longitudes_patron_SRM_ABE2 = hoja_Certificado_SRM2034["ABE = 0.5 nm"].dropna().values

# Se definen los valores de las bandas de %T mínima para un ABE = 1 nm desde el Certificado del NIST
longitudes_patron_SRM_ABE3 = hoja_Certificado_SRM2034["ABE = 1 nm"].values

# Se definen los valores de las bandas de %T mínima para un ABE = 2 nm desde el Certificado del NIST
longitudes_patron_SRM_ABE4 = hoja_Certificado_SRM2034["ABE = 2 nm"].values

In [39]:
# Se definen los arrays que contendrán las U certificadas del NIST de las bandas de transmitancia 
# mínima del SRM 2034

# Se definen los valores de las incertidumbres expandidas para un ABE = 0.1 nm desde el Certificado del NIST
U_longitudes_patron_SRM_ABE1 = hoja_Certificado_SRM2034["U.1 (nm)"].dropna().values

# Se definen los valores de las incertidumbres expandidas para un ABE = 0.5 nm desde el Certificado del NIST
U_longitudes_patron_SRM_ABE2 = hoja_Certificado_SRM2034["U.2 (nm)"].dropna().values

# Se definen los valores de las incertidumbres expandidas para un ABE = 1 nm desde el Certificado del NIST
U_longitudes_patron_SRM_ABE3 = hoja_Certificado_SRM2034["U.3 (nm)"].values

# Se definen los valores de las incertidumbres expandidas para un ABE = 2 nm desde el Certificado del NIST
U_longitudes_patron_SRM_ABE4 = hoja_Certificado_SRM2034["U.4 (nm)"].values

In [40]:
# Se definen los arrays que contendrán las desviaciones estándar de las bandas de transmitancia mínima 
# en función del ABE

# Se definen los valores de las desviaciones estándar para: ABE = 0.1 nm y RB = 14.91 nm/min 
s_SRM_Vel_Baja_ABE1 = hoja_Des_SRM_Vel_Baja["ABE = 0.1 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 0.5 nm y RB = 14.91 nm/min
s_SRM_Vel_Baja_ABE2 = hoja_Des_SRM_Vel_Baja["ABE = 0.5 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 1 nm y RB = 14.91 nm/min
s_SRM_Vel_Baja_ABE3 = hoja_Des_SRM_Vel_Baja["ABE = 1 nm"].values

# Se definen los valores de las desviaciones estándar para: ABE = 2 nm y RB = 14.91 nm/min
s_SRM_Vel_Baja_ABE4 = hoja_Des_SRM_Vel_Baja["ABE = 2 nm"].values

In [41]:
# Se definen los arrays que contendrán las incertidumbres combinadas máximas del primario en función del ABE
uComb_Max_Enf1_ABE1 = hoja_uComb_Primario_Max_Enf1["ABE = 0.1 nm"][0]
uComb_Max_Enf1_ABE2 = hoja_uComb_Primario_Max_Enf1["ABE = 0.5 nm"][0]
uComb_Max_Enf1_ABE3 = hoja_uComb_Primario_Max_Enf1["ABE = 1 nm"][0]
uComb_Max_Enf1_ABE4 = hoja_uComb_Primario_Max_Enf1["ABE = 2 nm"][0]

In [42]:
# Se definen los arrays que contendrán las incertidumbres combinadas máximas del primario en función del ABE
uComb_Max_Enf2_ABE1 = hoja_uComb_Primario_Max_Enf2["ABE = 0.1 nm"][0]
uComb_Max_Enf2_ABE2 = hoja_uComb_Primario_Max_Enf2["ABE = 0.5 nm"][0]
uComb_Max_Enf2_ABE3 = hoja_uComb_Primario_Max_Enf2["ABE = 1 nm"][0]
uComb_Max_Enf2_ABE4 = hoja_uComb_Primario_Max_Enf2["ABE = 2 nm"][0]

In [43]:
# Se definen los arrays que contendrán las correcciones para el Enf 1.1 
# que surgen del primario en función del ABE
Correccion_ABE1 = hoja_correcciones_SRM_Enf11["ABE = 0.1 nm"][0]
Correccion_ABE2 = hoja_correcciones_SRM_Enf11["ABE = 0.5 nm"][0]
Correccion_ABE3 = hoja_correcciones_SRM_Enf11["ABE = 1 nm"][0]
Correccion_ABE4 = hoja_correcciones_SRM_Enf11["ABE = 2 nm"][0]

#### Resultados de validación

##### ABE = 0.1 nm

In [ ]:
ABE = 0.1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

### Sub-Etapa 2: Análisis del enfoque 1, tomando en consideración un ajuste OLS a partir de los resultados del primario y la incertidumbre asociada a dicho ajuste.

La determinación de las longitudes de onda de transmitancia mínima del filtro de óxido de holmio SRM 2034, se obtiene mediante el siguiente 
modelo:

$$
\lambda_{EFAR_{i}} = \bar{\lambda_{i}}+C_{error}
$$
donde:
* $\lambda_{EFAR_{i}}$ es la medición del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $\bar{\lambda_{i}}$ es el promedio de las mediciones del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $C_{error}$ es la corrección originda por los datos del método primario de la primera etapa a partir de un ajuste lineal OLS

El diagrama de Ishikawa muestra las siguientes fuentes de incertidumbre para cada magnitud de entrada del modelo:

In [ ]:
Ishikawa_3 = r"C:\Users\Abraham Alfaro\Documents/TEC/2022/II SEMESTRE/TFG/Códigos en Jupyter para procesamiento de datos/Img/Ishikawa_VelBaja_1.2.PNG"
Image(filename = Ishikawa_3, width=500, height=500)

Para $\bar{\lambda_{i}}$ se tiene que:
$$
\boxed{u_{res} = \frac{d}{\sqrt{12}}}
$$

$$
d = 0,01\ \text{nm}
$$

$$
\boxed{u_{ABE} = \frac{s_{ponderada_{máxima}}}{\sqrt{3}}}
$$

$$
\boxed{u_{rep} = \frac{s_{\lambda_{i}}}{\sqrt{n}}}
$$
Para $C_{error}$ se tiene que:
$$
\boxed{u_{ajuste}=s_{ajuste}}
$$
donde:
$$
\boxed{s_{ajuste}=\sqrt{\dfrac{\sum(residuales)^2}{n-2}}}
$$

#### Importación de datos

In [47]:
# Se extraen los valores de las pendientes e intersección de los ajustes (Enf 1.2) en función del ABE
hoja_Parametros_OLS_SRM = pd.read_excel(path,sheet_name="Corrección SRM Enf1.2",header=0)
m_ABE1, b_ABE1, s_ajuste_ABE1 = hoja_Parametros_OLS_SRM["ABE = 0.1 nm"][0], hoja_Parametros_OLS_SRM["ABE = 0.1 nm"][1], hoja_Parametros_OLS_SRM["ABE = 0.1 nm"][2]
m_ABE2, b_ABE2, s_ajuste_ABE2 = hoja_Parametros_OLS_SRM["ABE = 0.5 nm"][0], hoja_Parametros_OLS_SRM["ABE = 0.5 nm"][1], hoja_Parametros_OLS_SRM["ABE = 0.5 nm"][2]
m_ABE3, b_ABE3, s_ajuste_ABE3 = hoja_Parametros_OLS_SRM["ABE = 1 nm"][0], hoja_Parametros_OLS_SRM["ABE = 1 nm"][1], hoja_Parametros_OLS_SRM["ABE = 1 nm"][2]
m_ABE4, b_ABE4, s_ajuste_ABE4 = hoja_Parametros_OLS_SRM["ABE = 2 nm"][0], hoja_Parametros_OLS_SRM["ABE = 2 nm"][1], hoja_Parametros_OLS_SRM["ABE = 2 nm"][2]

#### Resultados de la validación

##### ABE = 0.1

In [ ]:
ABE = 0.1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, s_ajuste_ABE1, m_ABE1, b_ABE1)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, s_ajuste_ABE2, m_ABE2, b_ABE2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, s_ajuste_ABE3, m_ABE3, b_ABE3)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, s_ajuste_ABE4, m_ABE4, b_ABE4)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

### Sub-etapa 3: cuantifica las longitudes de transmitancia mínima tomando en consideración el NO corregir, y el tomar la incertidumbre estádar combinada máxima de los errores de medida del ABE respectivo como la fuente de incertidumbre por el error primario del enfoque 2.

La determinación de las longitudes de onda de transmitancia mínima del filtro de óxido de holmio SRM 2034, se obtiene mediante el siguiente 
modelo:

$$
\lambda_{EFAR_{i}} = \bar{\lambda_{i}}+\epsilon_{error}
$$
donde:
* $\lambda_{EFAR_{i}}$ es la medición del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $\bar{\lambda_{i}}$ es el promedio de las mediciones del EFAR de la i-ésima banda de transmitancia mínima del filtro SRM 2034.
* $\epsilon_{error}$ es el aporte al modelo de la incertidumbre estándar combinada máxima del enfoque 2 del primario. Presenta media cero.

El diagrama de Ishikawa muestra las siguientes fuentes de incertidumbre para cada magnitud de entrada del modelo:

In [ ]:
Ishikawa_4 = r"C:\Users\Abraham Alfaro\Documents/TEC/2022/II SEMESTRE/TFG/Códigos en Jupyter para procesamiento de datos/Img/Ishikawa_VelBaja_2.PNG"
Image(filename = Ishikawa_4, width=500, height=500)

Para $\bar{\lambda_{i}}$ se tiene que:
$$
\boxed{u_{res} = \frac{d}{\sqrt{12}}}
$$

$$
d = 0,01\ \text{nm}
$$

$$
\boxed{u_{ABE} = \frac{s_{ponderada_{máxima}}}{\sqrt{3}}}
$$

$$
\boxed{u_{rep} = \frac{s_{\lambda_{i}}}{\sqrt{n}}}
$$
Para $\epsilon_{error}$ se tiene que:
$$
\boxed{u_{Error_{máx}}=u_{comb_{máx}}}
$$

#### Resultados de la validación

##### ABE = 0.1 nm

In [ ]:
ABE = 0.1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Baja(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Baja, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Baja, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

## Rapidez de barrido media: 29.63 nm/min

### Sub-etapa 1

#### Importación de datos

In [ ]:
# Se importan los datos en crudo necesarios para la validación de la hoja de cálculo en la segunda etapa

# Se importan los valores de longitudes promedio medidos del SRM a una velocidad media
hoja_longitudes_SRM_Vel_Med = pd.read_excel(path,sheet_name="Longitudes Promedio SRM VelMed",header=0)

# Se importan los valores de las correcciones para el enfoque 1.1 (Las mismas para todas las RB)
hoja_correcciones_SRM_Enf11 = pd.read_excel(path,sheet_name="Corrección SRM Enf1.1",header=0)

# Se importan los valores de las desviaciones estándar obtenidas con la RB media
hoja_Des_SRM_Vel_Med = pd.read_excel(path,sheet_name="s SRM 2034 Vel Med",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 1 (Primario) para Enf 1.1
hoja_uComb_Primario_Max_Enf1 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 1",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 2 (Primario) para Enf 2
hoja_uComb_Primario_Max_Enf2 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 2",header=0)

# Se importan los valores del certificado del SRM 2034
hoja_Certificado_SRM2034 = pd.read_excel(path,sheet_name="Certificado SRM 2034",header=0)

In [ ]:
# Se definen los arrays que contendrán los promedios mancomunados de las longitudes de onda de 
# transmitancia mínima en función del ABE

# Se definen las longitudes de onda promedio para: ABE = 0.1 nm y RB = 29.63 nm/min
longitudes_promedio_SRM_Vel_Med_ABE1 = hoja_longitudes_SRM_Vel_Med["ABE = 0.1 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 0.5 nm y RB = 29.63 nm/min
longitudes_promedio_SRM_Vel_Med_ABE2 = hoja_longitudes_SRM_Vel_Med["ABE = 0.5 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 1 nm y RB = 29.63 nm/min
longitudes_promedio_SRM_Vel_Med_ABE3 = hoja_longitudes_SRM_Vel_Med["ABE = 1 nm"].values

# Se definen las longitudes de onda promedio para: ABE = 2 nm y RB = 29.63 nm/min
longitudes_promedio_SRM_Vel_Med_ABE4 = hoja_longitudes_SRM_Vel_Med["ABE = 2 nm"].values

In [ ]:
# Se definen los arrays que contendrán las desviaciones estándar de las bandas de transmitancia mínima 
# en función del ABE

# Se definen los valores de las desviaciones estándar para: ABE = 0.1 nm y RB = 14.91 nm/min 
s_SRM_Vel_Med_ABE1 = hoja_Des_SRM_Vel_Med["ABE = 0.1 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 0.5 nm y RB = 14.91 nm/min
s_SRM_Vel_Med_ABE2 = hoja_Des_SRM_Vel_Med["ABE = 0.5 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 1 nm y RB = 14.91 nm/min
s_SRM_Vel_Med_ABE3 = hoja_Des_SRM_Vel_Med["ABE = 1 nm"].values

# Se definen los valores de las desviaciones estándar para: ABE = 2 nm y RB = 14.91 nm/min
s_SRM_Vel_Med_ABE4 = hoja_Des_SRM_Vel_Med["ABE = 2 nm"].values

#### Resultados de la validación

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

### Sub-etapa 2

#### Resultados de la validación

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, s_ajuste_ABE2, m_ABE2, b_ABE2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, s_ajuste_ABE3, m_ABE3, b_ABE3)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, s_ajuste_ABE4, m_ABE4, b_ABE4)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

### Sub-etapa 3

#### Resultados de la validación

##### ABE = 0.5 nm

In [ ]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

##### ABE = 1 nm

In [ ]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

##### ABE = 2 nm

In [ ]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Med(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Med, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Med, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

## Rapidez de barrido alta: 72.73 nm/min

### Sub-etapa 1

#### Importación de datos

In [17]:
# Se importan los datos en crudo necesarios para la validación de la hoja de cálculo en la segunda etapa

# Se importan los valores de longitudes promedio medidos del SRM a una velocidad alta
hoja_longitudes_SRM_Vel_Alt = pd.read_excel(path,sheet_name="Longitudes Promedio SRM VelAlt",header=0)

# Se importan los valores de las correcciones para el enfoque 1.1 (Las mismas para todas las RB)
hoja_correcciones_SRM_Enf11 = pd.read_excel(path,sheet_name="Corrección SRM Enf1.1",header=0)

# Se importan los valores de las desviaciones estándar obtenidas con la RB alta
hoja_Des_SRM_Vel_Alt = pd.read_excel(path,sheet_name="s SRM 2034 Vel Alt",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 1 (Primario) para Enf 1.1
hoja_uComb_Primario_Max_Enf1 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 1",header=0)

# Se importan los valores de las incertidumbres combinadas máximas del Enfoque 2 (Primario) para Enf 2
hoja_uComb_Primario_Max_Enf2 = pd.read_excel(path,sheet_name="uComb Primario Max Enf 2",header=0)

# Se importan los valores del certificado del SRM 2034
hoja_Certificado_SRM2034 = pd.read_excel(path,sheet_name="Certificado SRM 2034",header=0)

In [18]:
# Se definen los arrays que contendrán los promedios mancomunados de las longitudes de onda de 
# transmitancia mínima en función del ABE

# Se definen las longitudes de onda promedio para: ABE = 0.1 nm y RB = 72.73 nm/min
longitudes_promedio_SRM_Vel_Alt_ABE1 = hoja_longitudes_SRM_Vel_Alt["ABE = 0.1 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 0.5 nm y RB = 72.73 nm/min
longitudes_promedio_SRM_Vel_Alt_ABE2 = hoja_longitudes_SRM_Vel_Alt["ABE = 0.5 nm"].dropna().values

# Se definen las longitudes de onda promedio para: ABE = 1 nm y RB = 72.73 nm/min
longitudes_promedio_SRM_Vel_Alt_ABE3 = hoja_longitudes_SRM_Vel_Alt["ABE = 1 nm"].values

# Se definen las longitudes de onda promedio para: ABE = 2 nm y RB = 72.73 nm/min
longitudes_promedio_SRM_Vel_Alt_ABE4 = hoja_longitudes_SRM_Vel_Alt["ABE = 2 nm"].values

In [19]:
# Se definen los arrays que contendrán las desviaciones estándar de las bandas de transmitancia mínima 
# en función del ABE

# Se definen los valores de las desviaciones estándar para: ABE = 0.1 nm y RB = 14.91 nm/min 
s_SRM_Vel_Alt_ABE1 = hoja_Des_SRM_Vel_Alt["ABE = 0.1 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 0.5 nm y RB = 14.91 nm/min
s_SRM_Vel_Alt_ABE2 = hoja_Des_SRM_Vel_Alt["ABE = 0.5 nm"].dropna().values

# Se definen los valores de las desviaciones estándar para: ABE = 1 nm y RB = 14.91 nm/min
s_SRM_Vel_Alt_ABE3 = hoja_Des_SRM_Vel_Alt["ABE = 1 nm"].values

# Se definen los valores de las desviaciones estándar para: ABE = 2 nm y RB = 14.91 nm/min
s_SRM_Vel_Alt_ABE4 = hoja_Des_SRM_Vel_Alt["ABE = 2 nm"].values

#### Resultados de la validación

##### ABE = 0.5 nm

In [44]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.02,241.033815,0.003817,0.002887,0.001116,9.474567e-15,0.004174,0.005197,1.91,0.271
1,249.81,249.812814,0.003817,0.002887,0.001116,5.044382e-03,0.004174,0.007241,1.98,0.054
2,278.15,278.153813,0.003817,0.002887,0.001116,1.895102e-14,0.004174,0.005196,1.91,0.075
3,287.08,287.053817,0.003817,0.002887,0.001116,1.895030e-14,0.004174,0.005197,1.91,-0.513
4,333.47,333.481816,0.003817,0.002887,0.001116,1.999940e-03,0.004174,0.005569,1.93,0.231
5,345.43,345.438811,0.003817,0.002887,0.001116,5.425815e-03,0.004174,0.007515,1.98,0.169
6,361.27,361.285816,0.003817,0.002887,0.001116,1.333221e-03,0.004174,0.005365,1.92,0.309
7,385.45,385.490817,0.003817,0.002887,0.001116,5.382864e-03,0.004173,0.007480,1.98,0.956
8,416.07,416.093814,0.003817,0.002887,0.001116,1.894650e-14,0.004175,0.005197,1.91,0.466
9,467.80,467.783815,0.003817,0.002887,0.001116,3.791116e-14,0.004174,0.005195,1.91,-0.392


##### ABE = 1 nm

In [45]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.104093,0.004094,0.002887,0.001045,9.470943e-15,0.004585,0.005520,1.90,-0.311
1,249.89,249.888097,0.004094,0.002887,0.001045,4.989313e-03,0.004585,0.007441,1.97,-0.036
2,278.13,278.104091,0.004094,0.002887,0.001045,1.894237e-14,0.004586,0.005520,1.90,-0.506
3,287.22,287.204092,0.004094,0.002887,0.001045,1.894227e-14,0.004587,0.005521,1.90,-0.311
4,333.48,333.473094,0.004094,0.002887,0.001045,2.332923e-03,0.004585,0.005991,1.93,-0.134
5,345.38,345.397092,0.004094,0.002887,0.001045,4.724865e-03,0.004586,0.007266,1.97,0.328
6,361.25,361.254092,0.004094,0.002887,0.001045,0.000000e+00,0.004584,0.005517,1.90,0.080
7,385.61,385.599095,0.004094,0.002887,0.001045,3.729080e-03,0.004584,0.006660,1.96,-0.259
8,416.25,416.284095,0.004094,0.002887,0.001045,3.788631e-14,0.004584,0.005517,1.90,0.666
9,451.45,451.404095,0.004094,0.002887,0.001045,0.000000e+00,0.004586,0.005520,1.90,-0.897


##### ABE = 2 nm

In [46]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2,Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11 = Valida_Formulario_SRM_Enf11(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, Correccion)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf11,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf11, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, C_ABE, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_error, incertidumbre_estandar_comb_Enf11)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.015206,-0.014794,0.002887,0.002528,9.471800e-15,0.006322,0.007396,1.88,-2.010
1,250.03,249.946208,-0.014794,0.002887,0.002528,6.574616e-03,0.006324,0.009896,1.97,-1.558
2,278.10,277.985206,-0.014794,0.002887,0.002528,0.000000e+00,0.006322,0.007397,1.88,-2.202
3,287.52,287.405207,-0.014794,0.002887,0.002528,0.000000e+00,0.006324,0.007397,1.88,-2.202
4,333.47,333.385206,-0.014794,0.002887,0.002528,1.894696e-14,0.006322,0.007396,1.88,-1.626
5,345.42,345.353206,-0.014794,0.002887,0.002528,5.333461e-03,0.006324,0.009120,1.96,-1.255
6,361.12,361.085206,-0.014794,0.002887,0.002528,1.895383e-14,0.006323,0.007398,1.88,-0.667
7,385.80,385.735206,-0.014794,0.002887,0.002528,0.000000e+00,0.006323,0.007396,1.88,-1.519
8,416.57,416.555205,-0.014794,0.002887,0.002528,1.895451e-14,0.006321,0.007394,1.88,-0.284
9,451.32,451.235208,-0.014794,0.002887,0.002528,0.000000e+00,0.006323,0.007396,1.88,-1.988


### Sub-etapa 2

#### Resultados de la validación

##### ABE = 0.5 nm

In [48]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, s_ajuste_ABE2, m_ABE2, b_ABE2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.02,241.030524,0.000527,0.002887,0.001115,9.472500e-15,0.011016,0.011443,2.0,0.191
1,249.81,249.809727,0.000726,0.002887,0.001115,5.045196e-03,0.011013,0.012505,2.0,-0.005
2,278.15,278.151367,0.001367,0.002887,0.001115,1.894316e-14,0.011018,0.011444,2.0,0.025
3,287.08,287.051568,0.001569,0.002887,0.001115,1.895793e-14,0.011017,0.011445,2.0,-0.517
4,333.47,333.480614,0.002619,0.002887,0.001115,2.000181e-03,0.011017,0.011615,2.0,0.193
5,345.43,345.437891,0.002890,0.002887,0.001115,5.425570e-03,0.011014,0.012660,2.0,0.141
6,361.27,361.285252,0.003248,0.002887,0.001115,1.333746e-03,0.011013,0.011517,2.0,0.277
7,385.45,385.490792,0.003796,0.002887,0.001115,5.384834e-03,0.011014,0.012642,2.0,0.862
8,416.07,416.094483,0.004489,0.002887,0.001115,1.894248e-14,0.011017,0.011444,2.0,0.445
9,467.80,467.785653,0.005658,0.002887,0.001115,3.789839e-14,0.011017,0.011442,2.0,-0.311


##### ABE = 1 nm

In [50]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, s_ajuste_ABE3, m_ABE3, b_ABE3)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.103406,0.003416,0.002887,0.001045,9.472023e-15,0.010565,0.011003,2.0,-0.304
1,249.89,249.887454,0.003457,0.002887,0.001045,4.986972e-03,0.010564,0.012082,2.0,-0.046
2,278.13,278.103586,0.003589,0.002887,0.001045,1.894890e-14,0.010564,0.011002,2.0,-0.484
3,287.22,287.203633,0.003631,0.002887,0.001045,1.895144e-14,0.010568,0.011006,2.0,-0.300
4,333.48,333.472851,0.003847,0.002887,0.001045,2.333628e-03,0.010566,0.011246,2.0,-0.130
5,345.38,345.396901,0.003903,0.002887,0.001045,4.725603e-03,0.010564,0.011974,2.0,0.305
6,361.25,361.253973,0.003977,0.002887,0.001045,0.000000e+00,0.010566,0.011001,2.0,0.073
7,385.61,385.599094,0.004091,0.002887,0.001045,3.726981e-03,0.010563,0.011614,2.0,-0.236
8,416.25,416.284232,0.004234,0.002887,0.001045,3.788378e-14,0.010566,0.011002,2.0,0.627
9,451.45,451.404396,0.004398,0.002887,0.001045,0.000000e+00,0.010564,0.011000,2.0,-0.835


##### ABE = 2 nm

In [51]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE) 
longitud_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12 = Valida_Formulario_SRM_Enf12(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, s_ajuste_ABE4, m_ABE4, b_ABE4)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf12,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM_Corregida, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf12, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM_Corregida, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_ajuste, incertidumbre_estandar_comb_Enf12)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.012095,-0.017907,0.002887,0.002528,9.474286e-15,0.007847,0.008733,2.0,-2.037
1,250.03,249.943286,-0.017715,0.002887,0.002528,6.577589e-03,0.007848,0.010933,2.0,-1.589
2,278.10,277.982890,-0.017115,0.002887,0.002528,0.000000e+00,0.007849,0.008737,2.0,-2.211
3,287.52,287.403085,-0.016914,0.002887,0.002528,0.000000e+00,0.007849,0.008738,2.0,-2.207
4,333.47,333.384072,-0.015929,0.002887,0.002528,1.894612e-14,0.007852,0.008740,2.0,-1.622
5,345.42,345.352328,-0.015673,0.002887,0.002528,5.334187e-03,0.007845,0.010235,2.0,-1.253
6,361.12,361.084666,-0.015336,0.002887,0.002528,1.894666e-14,0.007846,0.008735,2.0,-0.667
7,385.80,385.735191,-0.014809,0.002887,0.002528,0.000000e+00,0.007849,0.008738,2.0,-1.485
8,416.57,416.555851,-0.014149,0.002887,0.002528,1.893631e-14,0.007852,0.008743,2.0,-0.267
9,451.32,451.236597,-0.013406,0.002887,0.002528,0.000000e+00,0.007850,0.008737,2.0,-1.911


### Sub-etapa 3

#### Resultados de la validación

##### ABE = 0.5 nm

In [53]:
ABE = 0.5
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.02,241.030004,0.0,0.002887,0.001115,9.472511e-15,0.044725,0.044819,2.0,0.097
1,249.81,249.809004,0.0,0.002887,0.001115,5.043123e-03,0.044725,0.045098,2.0,-0.010
2,278.15,278.149999,0.0,0.002887,0.001115,1.894995e-14,0.044725,0.044852,2.0,-0.000
3,287.08,287.050011,0.0,0.002887,0.001115,1.894342e-14,0.044725,0.044851,2.0,-0.292
4,333.47,333.477989,0.0,0.002887,0.001115,1.999553e-03,0.044725,0.044871,2.0,0.078
5,345.43,345.435010,0.0,0.002887,0.001115,5.427305e-03,0.044725,0.045144,2.0,0.049
6,361.27,361.281991,0.0,0.002887,0.001115,1.333078e-03,0.044725,0.044859,2.0,0.117
7,385.45,385.487005,0.0,0.002887,0.001115,5.386168e-03,0.044725,0.045163,2.0,0.375
8,416.07,416.089998,0.0,0.002887,0.001115,1.895324e-14,0.044725,0.044830,2.0,0.195
9,467.80,467.780003,0.0,0.002887,0.001115,3.790033e-14,0.044725,0.044804,2.0,-0.204


##### ABE = 1 nm

In [54]:
ABE = 1
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.099989,0.0,0.002887,0.001044,9.472250e-15,0.04302,0.043129,2.0,-0.201
1,249.89,249.884004,0.0,0.002887,0.001044,4.986642e-03,0.04302,0.043397,2.0,-0.060
2,278.13,278.100018,0.0,0.002887,0.001044,1.894553e-14,0.04302,0.043126,2.0,-0.301
3,287.22,287.199971,0.0,0.002887,0.001044,1.894772e-14,0.04302,0.043148,2.0,-0.201
4,333.48,333.468998,0.0,0.002887,0.001044,2.334039e-03,0.04302,0.043189,2.0,-0.110
5,345.38,345.393008,0.0,0.002887,0.001044,4.724934e-03,0.04302,0.043368,2.0,0.130
6,361.25,361.250013,0.0,0.002887,0.001044,0.000000e+00,0.04302,0.043123,2.0,0.000
7,385.61,385.594994,0.0,0.002887,0.001044,3.727009e-03,0.04302,0.043287,2.0,-0.157
8,416.25,416.280019,0.0,0.002887,0.001044,3.789552e-14,0.04302,0.043130,2.0,0.301
9,451.45,451.399982,0.0,0.002887,0.001044,0.000000e+00,0.04302,0.043138,2.0,-0.502


##### ABE = 2 nm

In [55]:
ABE = 2
longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf1, uComb_Max_Enf2, Correccion = Selecciona_Parametros_SRM_Vel_Alt(ABE)
longitud_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2 = Valida_Formulario_SRM_Enf2(longitudes_patron_SRM, longitudes_promedio_SRM_Vel_Alt, u_ABE, U_longitudes_patron_SRM, s_SRM_Vel_Alt, uComb_Max_Enf2)
k = Calculo_k(longitud_SRM,incertidumbre_estandar_comb_Enf2,0.9545)
E_N = E_Normalizado(longitud_promedio_SRM, longitudes_patron_SRM, incertidumbre_estandar_comb_Enf2, U_longitudes_patron_SRM)
df_final = DataFrame_Final_SRM(E_N, k, longitudes_patron_SRM, longitud_promedio_SRM, CorreccionAjuste, incertidumbre_estandar_res, incertidumbre_estandar_ABE, incertidumbre_estandar_rep, incertidumbre_estandar_u_comb_max_enf2, incertidumbre_estandar_comb_Enf2)
df_final

,Longitud de onda de ref (nm),Longitud de onda EFAR (nm),Corrección,$u_{res}$ (nm),$u_{ABE}$ (nm),$u_{rep}$ (nm),$u_{error}$ (nm),$u_{c}$ (nm),"Factor de cobertura, k",$E_{N}$
0,241.12,241.030004,0.0,0.002887,0.002528,9.472672e-15,0.068235,0.068350,2.0,-0.618
1,250.03,249.961008,0.0,0.002887,0.002528,6.572733e-03,0.068235,0.068643,2.0,-0.472
2,278.10,277.999985,0.0,0.002887,0.002528,0.000000e+00,0.068235,0.068368,2.0,-0.687
3,287.52,287.419955,0.0,0.002887,0.002528,0.000000e+00,0.068235,0.068334,2.0,-0.687
4,333.47,333.400013,0.0,0.002887,0.002528,1.894823e-14,0.068235,0.068341,2.0,-0.481
5,345.42,345.368010,0.0,0.002887,0.002528,5.332241e-03,0.068235,0.068544,2.0,-0.356
6,361.12,361.100006,0.0,0.002887,0.002528,1.894676e-14,0.068235,0.068345,2.0,-0.137
7,385.80,385.749987,0.0,0.002887,0.002528,0.000000e+00,0.068235,0.068327,2.0,-0.351
8,416.57,416.569961,0.0,0.002887,0.002528,1.895403e-14,0.068235,0.068341,2.0,-0.000
9,451.32,451.249979,0.0,0.002887,0.002528,0.000000e+00,0.068235,0.068349,2.0,-0.492


# FIN DE LA VALIDACIÓN